In [11]:
# importing for ipywidgets
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [12]:
#import to filter warnings
import warnings
warnings.filterwarnings('ignore')
#!pip install numpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from matplotlib.pyplot import xticks
%matplotlib inline
sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(16,10)})
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', -1)

In [3]:
#### convert the next cell to code to install textblob

!pip install -U textblob

In [21]:

import calendar
import datetime as dt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('movie_reviews')

from nltk.tokenize import word_tokenize

from nltk.tokenize import RegexpTokenizer
from collections import Counter
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\twitter_samples.zip.


# Put your input in the next block and name it inputdf

In [5]:

inputdf=pd.read_excel("try.xlsx",encoding="utf-8")


In [ ]:
inputdf['comments'] = inputdf['comments_text'].apply(lambda x: ','.join(map(str, x)))
inputdf

In [6]:
@interact
def readingcomments(video=inputdf.video_title.unique()):
    words = inputdf.loc[inputdf['video_title'] == video].comments[0].lower().split(sep="b'")
    textblock=" ".join(words)
    wordsdf= pd.DataFrame(columns=["Comments_on_video ---   " +video],data=words)
    return wordsdf

interactive(children=(Dropdown(description='video', options=('Maithili Thakur sings her version of "Perfect"',…

In [7]:
global Sentiment
Heads=["Post_Id",'Video_Title', 'Sentiment on Video','Subjectivity on Video']
Sentiment=pd.DataFrame(columns=Heads)
Sentiment

,Post_Id,Video_Title,Sentiment on Video,Subjectivity on Video


In [8]:
@interact
def readingcomments(video=inputdf.video_title.unique()):
    comments_words = inputdf.loc[inputdf['video_title'] == video].comments.str.lower().str.cat(sep=' ').split()
    comments_as_text_block=" ".join(comments_words)
    tokenizer = RegexpTokenizer(r'\w+')
    result = tokenizer.tokenize(comments_as_text_block)
    text = " ".join(result)
    tokens_stopwords = word_tokenize(text)
    no_stopwords_tokens = [word for word in tokens_stopwords if not word in stopwords.words()]
    #print(no_stopwords_tokens)
    #print(text_tokens)
    textblock_without_stopwords=" ".join(no_stopwords_tokens)
    textblock_with_stopwords=" ".join(tokens_stopwords)
    top_N = 1000
    word_all = pd.DataFrame(Counter(no_stopwords_tokens).most_common(top_N),
                    columns=['word', 'Freq']).set_index('word')
    word_count=word_all[word_all.Freq>1]
    #mask = np.array(Image.open('E:/Insights/logos/cloud.jpg'))
    #image_colors = ImageColorGenerator(mask)
    #wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white",mask=mask,color_func = image_colors,contour_width=7, contour_color='blue').generate(textblock_without_stopwords)
    #fig=plt.figure(figsize=(12,10))
    #plt.imshow(wordcloud, interpolation="bilinear")
    #plt.axis("off")
    #plt.tight_layout(pad=0)
    wordcloud = WordCloud(max_font_size=50, max_words=30, background_color="white").generate(textblock_without_stopwords)
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()
    sent          = TextBlob(textblock_without_stopwords)
    polarity      = sent.sentiment.polarity
    subjectivity  = sent.sentiment.subjectivity
    sent          = TextBlob(text, analyzer = NaiveBayesAnalyzer())
    classification= sent.sentiment.classification
    positive      = sent.sentiment.p_pos
    negative      = sent.sentiment.p_neg
    if (polarity <0.2 and polarity >= 0):
        Senti="Neutral"
    elif (polarity > 0.2):
        Senti="Positive"
    else:
        Senti="Negative"
    if (subjectivity <0.2 and subjectivity >= 0):
        Senti="Neutral"
    elif (polarity > 0.2):
        Senti="Positive"
    else:
        Senti="Negative"
    if (subjectivity <0.2):
        Subj="Highly Objective"
    elif (subjectivity >0.2 and subjectivity >= 0.4):
        Subj="Slightly Objective"
    elif (subjectivity >0.4 and subjectivity >= 0.6):
        Subj="Subjective"
    else:
        Subj="Highly Subjective"
    #global Sentiment
    #Heads=[" Post_id", 'Video_Title', 'Sentiment on Video','Subjectivity on Video']
    #Sentiment=pd.DataFrame(columns=Heads)
    #rows =[[video, Senti, Subj]]
    rows =[[len(Sentiment),video, Senti, Subj]]
    #new_row = pd.Series({'Video_Title': video, 'Sentiment on Video': Senti, 'Subjectivity on Video': Subj})
    for each_video_result in rows:
             Sentiment.loc[len(Sentiment)] = each_video_result
    Sentiment.set_index("Post_Id")
    return plt.show(), print(Sentiment)

interactive(children=(Dropdown(description='video', options=('Maithili Thakur sings her version of "Perfect"',…

In [10]:
Sentiment

,Post_Id,Video_Title,Sentiment on Video,Subjectivity on Video
0,0,"Maithili Thakur sings her version of ""Perfect""",Positive,Slightly Objective
1,1,Zomato,Negative,Slightly Objective
